<a href="https://colab.research.google.com/github/satvik959/DL-2025-26/blob/main/WEEK4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#USING OPTIMIZATION ALGO
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import altair as alt
import tensorflow as tf # Import TensorFlow

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# expected values
y = np.array([[0, 1, 1, 0]]).T

# features
X = np.array([[0, 0, 1, 1],
              [0, 1, 0, 1]]).T


model = Sequential()
model.add(Dense(16, input_dim=2, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['binary_accuracy', 'mean_squared_error'])

history = model.fit(X, y, epochs=750, verbose=0)


errors = history.history['loss']

df2 = pd.DataFrame({"errors":errors, "time-step": np.arange(0, len(errors))})

alt.Chart(df2).mark_line().encode(x="time-step", y="errors").properties(title='Chart 3')


y_pred = model.predict(X).round()
num_correct_predictions = (y_pred == y).sum()
accuracy = (num_correct_predictions / y.shape[0]) * 100
print('Multi-layer perceptron accuracy: %.2f%%' % accuracy)





/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Multi-layer perceptron accuracy: 100.00%


In [2]:
# Get accuracy history
accuracy_history = history.history['binary_accuracy']

# Create DataFrame for accuracy
df_acc = pd.DataFrame({
    "accuracy": accuracy_history,
    "epoch": np.arange(1, len(accuracy_history) + 1)
})

# Plot accuracy over epochs
alt.Chart(df_acc).mark_line().encode(
    x="epoch",
    y="accuracy"
).properties(
    title='Model Accuracy Over Epochs'
)

alt.Chart(...)

In [3]:
# USING OPTIMIZATION ALGO WITH EARLY STOP AT 100% ACCURACY
import numpy as np
import pandas as pd
import altair as alt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import Callback

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# XOR dataset
y = np.array([[0, 1, 1, 0]]).T
X = np.array([[0, 0, 1, 1],
              [0, 1, 0, 1]]).T


# Custom callback to stop at 100% accuracy
class StopAtAccuracy(Callback):
    def __init__(self, target_accuracy=1.0):
        super(StopAtAccuracy, self).__init__()
        self.target_accuracy = target_accuracy

    def on_epoch_end(self, epoch, logs=None):
        acc = logs.get("binary_accuracy")
        if acc >= self.target_accuracy:
            print(f"\nReached {self.target_accuracy*100}% accuracy. Stopping training.")
            self.model.stop_training = True


# Build model
model = Sequential()
model.add(Dense(16, input_dim=2, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['binary_accuracy'])

# Train model
history = model.fit(
    X, y,
    epochs=10000,   # Large upper limit
    verbose=0,
    callbacks=[StopAtAccuracy(target_accuracy=1.0)]
)

# ----------------------
# Plot Loss Graph
# ----------------------
df_loss = pd.DataFrame({
    "Loss": history.history['loss'],
    "Epoch": np.arange(1, len(history.history['loss']) + 1)
})

loss_chart = alt.Chart(df_loss).mark_line().encode(
    x='Epoch',
    y='Loss'
).properties(title='Loss vs Epochs')

loss_chart

# ----------------------
# Plot Accuracy Graph
# ----------------------
df_acc = pd.DataFrame({
    "Accuracy (%)": np.array(history.history['binary_accuracy']) * 100,
    "Epoch": np.arange(1, len(history.history['binary_accuracy']) + 1)
})

accuracy_chart = alt.Chart(df_acc).mark_line(color='green').encode(
    x='Epoch',
    y='Accuracy (%)'
).properties(title='Accuracy vs Epochs')

accuracy_chart

# ----------------------
# Final Predictions
# ----------------------
y_pred = model.predict(X).round()
num_correct_predictions = (y_pred == y).sum()
accuracy = (num_correct_predictions / y.shape[0]) * 100

print('\nMulti-layer perceptron accuracy: %.2f%%' % accuracy)
print("Training stopped at epoch:", len(history.history['loss']))

# ----------------------
# Display Weights and Biases
# ----------------------
print("\nModel Weights and Biases at 100% Accuracy:\n")

for i, layer in enumerate(model.layers):
    weights, biases = layer.get_weights()
    print(f"Layer {i+1}")
    print("Weights:\n", weights)
    print("Biases:\n", biases)
    print("-" * 50)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Reached 100.0% accuracy. Stopping training.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step

Multi-layer perceptron accuracy: 100.00%
Training stopped at epoch: 333

Model Weights and Biases at 100% Accuracy:

Layer 1
Weights:
 [[-0.74299896  0.52893066  0.35173583  0.35466802 -1.0598795  -0.72550565
  -0.96630436  0.07064448  0.8783171   0.50318205 -0.12989506 -0.21981098
   0.5015953   0.4773172   0.8461785  -0.1341153 ]
 [ 0.39778224 -0.22091015  0.51128966  0.42681983  0.65925467 -0.85781777
  -0.9163604   0.2815984   0.95563227 -0.15775545  0.40559718 -0.45016193
  -0.771892   -0.7117484   0.6767954  -0.5689938 ]]
Biases:
 [-0.4461509   0.37499315 -0.4920391  -0.44171417 -0.55388933 -0.4427902
 -0.367211    0.00590088  0.35818285  0.32523987  0.2192489   0.28465402
 -0.4208252  -0.49931124  0.44844294  0.2977826 ]
--------------------------------------------------
Layer 2
Weights:
 [[ 0.54753876]
 [-0.74873286]
 [-0.45857742]
 [-0.36307266]
 [ 0.5635146 ]
 [-0.7576964 ]
 [-0.8426965 ]
 [-0

In [1]:
import numpy as np
import pandas as pd
import altair as alt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import (
    SGD, RMSprop, Adam,
    Adagrad, Adadelta, Adamax, Nadam, Ftrl
)


# Set seeds
np.random.seed(42)
tf.random.set_seed(42)

# Data (XOR problem)
y = np.array([[0, 1, 1, 0]]).T
X = np.array([[0, 0, 1, 1],
              [0, 1, 0, 1]]).T

# List of optimizers to compare
optimizers = {
    # Basic SGD
    "SGD": SGD(learning_rate=0.1),

    # SGD with Momentum
    "SGD_Momentum": SGD(learning_rate=0.1, momentum=0.9),

    # Nesterov Momentum
    "SGD_Nesterov": SGD(learning_rate=0.1, momentum=0.9, nesterov=True),

    # Adaptive methods
    "Adagrad": Adagrad(learning_rate=0.1),
    "Adadelta": Adadelta(learning_rate=1.0),
    "RMSprop": RMSprop(learning_rate=0.01),

    # Adam family
    "Adam": Adam(learning_rate=0.01),
    "Adamax": Adamax(learning_rate=0.01),
    "Nadam": Nadam(learning_rate=0.01),

    # FTRL (mainly used for sparse models, but can test)
    "Ftrl": Ftrl(learning_rate=0.1)
}


results = []

# Train model with each optimizer
for name, opt in optimizers.items():

    # Recreate model each time
    model = Sequential()
    model.add(Dense(16, input_dim=2, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='mean_squared_error',
                  optimizer=opt,
                  metrics=['binary_accuracy'])

    history = model.fit(X, y, epochs=300, verbose=0)

    # Store accuracy history
    for epoch, acc in enumerate(history.history['binary_accuracy']):
        results.append({
            "epoch": epoch,
            "accuracy": acc * 100,
            "optimizer": name
        })

# Create DataFrame
df = pd.DataFrame(results)

# Plot comparison graph
chart = alt.Chart(df).mark_line().encode(
    x=alt.X('epoch', title='Epoch'),
    y=alt.Y('accuracy', title='Accuracy (%)', scale=alt.Scale(domain=[0, 100])),
).properties(
    width=200,
    height=150
).facet(
    facet='optimizer',
    columns=3
).properties(
    title='Accuracy Comparison of Optimizers (XOR)'
)

chart



/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


alt.FacetChart(...)